In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [40]:
class Fields():
    
    def __init__(self, l = 2, N = 11):
        # Domain is a cube with N^3 points
        self.l   = l
        self.h   = l / N
        self.N   = N
        
        self.comp_r()
        #Staggered Grid:
        self.r -= 0.5*self.h

        self.A_new,  self.A   = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.B_new,  self.B   = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.DA_new, self.DA  = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.DB_new, self.DB  = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.KA_new, self.KA  = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.KB_new, self.KB  = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.Dal_new,self.Dal = np.zeros((N,N,N)), np.zeros((N,N,N))
        self.al_new, self.al  = np.zeros((N,N,N)), np.zeros((N,N,N))
        
    def comp_r(self):
        """
        Compute the distance from the center. MODIFY!!!!!!
        """
        self.r = np.zeros((self.N,self.N,self.N))
        for i in range(self.N):
            for j in range(self.N):
                for k in range(self.N):
                    x = - self.l/2 + i * self.h
                    y = - self.l/2 + j * self.h
                    z = - self.l/2 + k * self.h
                    self.r[i,j,k] = np.sqrt(x**2 + y**2 + z**2)
    
    def IC_geodesicSlicing(self, M = 1):
        """
        Geodesic Slicing gauge condition.
                A = B = \psi = 1 + M /(2*r)
                KA = KB = 0
                alpha = 1 = costant
        """
        self.A  = 1 + 0.5 * M / self.r
        self.B  = 1 + 0.5 * M / self.r
        for i in range(self.N):
            for j in range(self.N):
                for k in range(self.N):
                    if i == 0:
                        self.DA[i,j,k] = 
                        self.DB[i,j,k] =
                    
                    
                    
                    else: # General case
                        self.DA[i,j,k] = der_r(np.log(self.A), i,j,k, self.h, self.l)
                        self.DB[i,j,k] = der_r(np.log(self.B), i,j,k, self.h, self.l)
        
        
        
        
        self.DA_Bound = np.zeros((self.N, self.N, self.N))
        self.DB_Bound = np.zeros((self.N, self.N, self.N))
        
        self.DA_Bound[0,:,:],self.DA_Bound[-1,:,:] = self.DA[0,:,:], self.DA[-1,:,:]
        self.DA_Bound[:,0,:],self.DA_Bound[:,-1,:] = self.DA[:,0,:], self.DA[:,-1,:]
        self.DA_Bound[:,:,0],self.DA_Bound[:,:,-1] = self.DA[:,:,0], self.DA[:,:,-1]
        
        self.DB_Bound[0,:,:],self.DB_Bound[-1,:,:] = self.DB[0,:,:], self.DB[-1,:,:]
        self.DB_Bound[:,0,:],self.DB_Bound[:,-1,:] = self.DB[:,0,:], self.DB[:,-1,:]
        self.DB_Bound[:,:,0],self.DB_Bound[:,:,-1] = self.DB[:,:,0], self.DB[:,:,-1]
        
        self.al += 1
        
    def IC_1plusLogSlicing(self, M = 1):
        pass
    
    def a_symmetryCond(self):
        cc = int(np.floor(self.N/2))
        self.A  [cc,cc,cc] = self.A  [cc+1,cc,cc]
        self.B  [cc,cc,cc] = self.B  [cc+1,cc,cc]
        self.DA [cc,cc,cc] = self.DA [cc+1,cc,cc]
        self.DB [cc,cc,cc] = self.DB [cc+1,cc,cc]
        self.KA [cc,cc,cc] = self.KA [cc+1,cc,cc]
        self.KB [cc,cc,cc] = self.KB [cc+1,cc,cc]
        self.Dal[cc,cc,cc] = self.Dal[cc+1,cc,cc]
        self.al [cc,cc,cc] = self.al [cc+1,cc,cc]
    
    def BC_Dirichlet(self, M = 1):
        self.A[0,:,:], self.A[-1,:,:] = 1 + 0.5 * M / self.r[0,:,:], 1 + 0.5 * M / self.r[-1,:,:]
        self.A[:,0,:], self.A[:,-1,:] = 1 + 0.5 * M / self.r[:,0,:], 1 + 0.5 * M / self.r[:,-1,:]
        self.A[:,:,0], self.A[:,:,-1] = 1 + 0.5 * M / self.r[:,:,0], 1 + 0.5 * M / self.r[:,:,-1]
        
        self.B[0,:,:], self.B[-1,:,:] = 1 + 0.5 * M / self.r[0,:,:], 1 + 0.5 * M / self.r[-1,:,:]
        self.B[:,0,:], self.B[:,-1,:] = 1 + 0.5 * M / self.r[:,0,:], 1 + 0.5 * M / self.r[:,-1,:]
        self.B[:,:,0], self.B[:,:,-1] = 1 + 0.5 * M / self.r[:,:,0], 1 + 0.5 * M / self.r[:,:,-1]
        
        self.DA[0,:,:], self.DA[-1,:,:] = self.DA_Bound[0,:,:],self.DA_Bound[-1,:,:]
        self.DA[:,0,:], self.DA[:,-1,:] = self.DA_Bound[:,0,:],self.DA_Bound[:,-1,:]
        self.DA[:,:,0], self.DA[:,:,-1] = self.DA_Bound[:,:,0],self.DA_Bound[:,:,-1]
        
        self.DB[0,:,:], self.DB[-1,:,:] = self.DB_Bound[0,:,:],self.DB_Bound[-1,:,:]
        self.DB[:,0,:], self.DB[:,-1,:] = self.DB_Bound[:,0,:],self.DB_Bound[:,-1,:]
        self.DB[:,:,0], self.DB[:,:,-1] = self.DB_Bound[:,:,0],self.DB_Bound[:,:,-1]

        self.KA[0,:,:], self.KA[-1,:,:] = 0, 0
        self.KA[:,0,:], self.KA[:,-1,:] = 0, 0
        self.KA[:,:,0], self.KA[:,:,-1] = 0, 0
        
        self.KB[0,:,:], self.KB[-1,:,:] = 0, 0
        self.KB[:,0,:], self.KB[:,-1,:] = 0, 0
        self.KB[:,:,0], self.KB[:,:,-1] = 0, 0
        
        self.Dal[0,:,:], self.Dal[-1,:,:] = 0, 0
        self.Dal[:,0,:], self.Dal[:,-1,:] = 0, 0
        self.Dal[:,:,0], self.Dal[:,:,-1] = 0, 0

# RHS of eqs (3)
These are decomposed using finite differences.

In [41]:
def to_coord(i,j,k,h,l):
    x = - l/2 + i * h
    y = - l/2 + j * h
    z = - l/2 + k * h
    return x,y,z

def angs(i,j,k,h,l):
    x,y,z = to_coord(i,j,k,h,l)
    if np.sqrt(x**2 + y**2 + z**2) > 1e-4:
        th = np.arccos(z / np.sqrt(x**2 + y**2 + z**2))
    else:
        th = 0
    ph = np.arctan2(y, x)
    return th, ph

def der_r(f,i,j,k,h,l):
    """
    TO DO:: Pass only the cube that contains the data needed, not everything
    
    f      :: field to derive wrt the radial direction
    i,j,k  :: position in the grid
    h      :: spacing of the grid
    l      :: length of the domain (1D)
    """
    th, ph = angs(i,j,k,h,l)

    out1 = np.cos(ph) * np.sin(th) * (f[i+1,j,k] - f[i-1,j,k]) ## x derivative
    out2 = np.sin(ph) * np.sin(th) * (f[i,j+1,k] - f[i,j-1,k]) ## y derivative
    out3 =              np.cos(th) * (f[i,j,k+1] - f[i,j,k-1]) ## z derivative
    
    return (out1 + out2 + out3) * 0.5 / h

In [42]:
def ev_A(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i,j,k] * field.A[i,j,k] * field.KA[i,j,k]

def ev_B(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i,j,k] * field.B[i,j,k] * field.KB[i,j,k]

def ev_DA(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KA[i,j,k] * field.Dal[i,j,k]
    p2 = der_r(field.KA, i,j,k, field.h, field.l)
    return - 2 * field.al[i,j,k] * (p1 + p2)

def ev_DB(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KB[i,j,k] * field.Dal[i,j,k]
    p2 = der_r(field.KB, i,j,k, field.h, field.l)
    return - 2 * field.al[i,j,k] * (p1 + p2)


def ev_KA(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    x,y,z = to_coord(i,j,k, field.h, field.l)
    r = np.sqrt(x**2 + y**2 + z**2)
    r -= 0.5*field.h
    
    p1 = der_r(field.Dal + field.DB, i,j,k, field.h, field.l)
    p2 = field.Dal[i,j,k]**2 + 0.5 * (field.Dal[i,j,k] * field.DA[i,j,k] + field.DB[i,j,k]**2 - field.DA[i,j,k] * field.DB[i,j,k])
    p3 = - field.A[i,j,k] * field.KA[i,j,k] * (field.KA[i,j,k] + 2*field.KB[i,j,k])
    p4 = - (field.DA[i,j,k] - 2 * field.DB[i,j,k]) / r
    return - field.al[i,j,k] * (p1 + p2 + p3 + p4) / field.A[i,j,k]
    
def ev_KB(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    x,y,z = to_coord(i,j,k, field.h, field.l)
    r = np.sqrt(x**2 + y**2 + z**2)
    r -= 0.5*field.h
    
    p1 = der_r(field.DB, i,j,k, field.h, field.l)
    p2 = field.Dal[i,j,k] * field.DB[i,j,k] + field.DB[i,j,k]**2 - 0.5 * field.DA[i,j,k] * field.DB[i,j,k]
    p3 = - (field.DA[i,j,k] - 2 * field.Dal[i,j,k] - 4 * field.DB[i,j,k]) / r
    p4 = - 2 * (field.A[i,j,k] - field.B[i,j,k]) / (field.B[i,j,k] * r**2)
    p5 = field.al[i,j,k] * field.KB[i,j,k] * (field.KA[i,j,k] + 2*field.KB[i,j,k])
    return - 0.5 * field.al[i,j,k] * (p1+p2+p3+p4) / field.A[i,j,k] + p5
    

# Time evolution

In [ ]:
dt = 0.01
t = 0
N = 11

fields = Fields(l = 2, N = N)
fields.IC_geodesicSlicing()

cc = np.floor(fields.N / 2)

for _ in range(1):
    t += dt
    for i in range(1,N-1):
        for j in range(1,N-1):
            for k in range(1,N-1):
                if i == cc and j == cc and k == cc:
                    continue
                
                #fields.a_symmetryCond()
                fields.BC_Dirichlet()
                
                fields.A_new [i, j, k] = fields.A [i, j, k] + dt * ev_A (fields, i, j, k)
                fields.B_new [i, j, k] = fields.B [i, j, k] + dt * ev_B (fields, i, j, k)
                fields.DA_new[i, j, k] = fields.DA[i, j, k] + dt * ev_DA(fields, i, j, k)
                fields.DB_new[i, j, k] = fields.DB[i, j, k] + dt * ev_DB(fields, i, j, k)
                fields.KA_new[i, j, k] = fields.KA[i, j, k] + dt * ev_KA(fields, i, j, k)
                fields.KB_new[i, j, k] = fields.KB[i, j, k] + dt * ev_KB(fields, i, j, k)
                
                
                
                fields.A  = fields.A_new
                fields.B  = fields.B_new
                fields.DA = fields.DA_new
                fields.DB = fields.DB_new
                fields.KA = fields.KA_new
                fields.KB = fields.KB_new
                

In [ ]:
print(fields.B)

In [52]:
test = Fields()
test.IC_geodesicSlicing()
test.DA

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -0.2041861 , -0.23229229, ..., -0.25965087,
         -0.23229229,  0.        ],
        [ 0.        , -0.23229229, -0.2695913 , ..., -0.30757934,
         -0.2695913 ,  0.        ],
        ...,
        [ 0.        , -0.25965087, -0.30757934, ..., -